In [1]:
from tools import SL2P
from tools import read_sentinel2_safe_image
from tools.read_sentinel2_force_image import read_s2_force
from tools import dictionariesSL2P 
import os,numpy
import rasterio

In [ ]:
### FORCE seperate Bands + S3 Angles ###

In [37]:
variableName = ''
# LAI ; fCOVER ; fAPAR ; CCC ; CWC ; Albedo
imageCollectionName = "S2_FORCE" 
tile_dir = r'C:\Users\Leoun\Work\SL2P-PYTHON-main\FORCE'

In [38]:
# check variable and collection
if variableName not in dictionariesSL2P.make_net_options().keys():
    raise ValueError('Supported vegetation variables: %s' % (dictionariesSL2P.make_net_options().keys()))
if imageCollectionName not in dictionariesSL2P.make_net_options()[variableName].keys():
    raise ValueError('Supported EO datasets: %s' % (dictionariesSL2P.make_net_options()[variableName].keys()))

In [39]:
# read FORCE tile
s2 = read_s2_force(tile_dir)  # replaces read_sentinel2_safe_image.read_s2

In [40]:
# prepare SL2P input
sl2p_inp = SL2P.prepare_sl2p_inp(s2, variableName, imageCollectionName)

Skipping resampling: Assuming FORCE tile angles are matched to S2_FORCE resolution.
Computing cosSZA, cosVZA and cosRAA
Scaling Sentinel-2 bands
Selecting sl2p input bands
Done!


In [41]:
varmap=SL2P.SL2P(sl2p_inp,variableName,imageCollectionName)

Generating sl2p input data flag
Run SL2P...
SL2P start: 2025-10-22 11:36:50.226181
SL2P end: 2025-10-22 11:36:51.520517
Generating sl2p output product flag
Done


In [42]:
# Define the output path variables (must be present in your namespace)
output_dir = r'C:\Users\Leoun\Work\SL2P-FORCE-main\output'
tile_basename = os.path.basename(os.path.normpath(tile_dir))
imageName = os.path.join(output_dir, tile_basename) # Create a base name to replace the SAFE filename

In [43]:
profile = s2['profile'].copy()
profile.update({
    'count': 4, 
    'dtype': rasterio.float32, # Use float32 for all 4 bands
    'driver': 'GTiff'
}) 

flag_inp = varmap['sl2p_inputFlag'].astype(numpy.float32)
flag_out = varmap['sl2p_outputFlag'].astype(numpy.float32)

output_path = imageName + f"_{variableName}_PRODUCTS.tif"

# 4. Write all 4 layers
with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write(varmap[variableName].astype(numpy.float32), 1)
    dst.write(varmap[variableName+'_uncertainty'].astype(numpy.float32), 2)
    dst.write(flag_inp, 3) # Write the converted input flag
    dst.write(flag_out, 4)  # Write the converted output flag

print(f"✅ 4-Band product saved successfully to: {output_path}")

✅ 4-Band product saved successfully to: C:\Users\Leoun\Work\SL2P-FORCE-main\output\new\FORCE_Albedo_PRODUCTS.tif


In [ ]:
### Single FORCE TIF + S3 Angles ###

In [16]:
# Ensure you import the new function name
from tools.read_sentinel2_force_image import read_s2_force, read_single_tif_s2 
from tools import read_sentinel2_safe_image # Keep this base import if needed elsewhere

In [45]:
variableName = ''
# LAI ; fCOVER ; fAPAR ; CCC ; CWC ; Albedo
imageCollectionName = "S2_SINGLE_TIF" # *** NEW MODE ***

In [46]:
# 1. Path to the multi-band TIF 
single_tif_path = r'C:\Users\Leoun\Work\safe_equi\20190726_LEVEL2_SEN2A_BOA.tif'

# 2. Path to the FORCE S3 angletiles
angle_tile_dir = r'C:\Users\Leoun\Work\SL2P-PYTHON-main\FORCE'

# Define base name for output
tile_basename = os.path.splitext(os.path.basename(single_tif_path))[0]

In [47]:
print(f"Reading TIF bands: {single_tif_path}")
print(f"Reading SAFE angles: {angle_tile_dir}")

# *** NEW HYBRID READER CALL ***
s2 = read_single_tif_s2(single_tif_path, angle_tile_dir)

Reading TIF bands: C:\Users\Leoun\Work\safe_equi\20190726_LEVEL2_SEN2A_BOA.tif
Reading SAFE angles: C:\Users\Leoun\Work\SL2P-PYTHON-main\FORCE


In [48]:
# prepare SL2P input
sl2p_inp = SL2P.prepare_sl2p_inp(s2, variableName, imageCollectionName)

Resample sun and view (sensor) angles for SAFE format
Computing cosSZA, cosVZA and cosRAA
Scaling Sentinel-2 bands
Selecting sl2p input bands
Done!


In [49]:
varmap=SL2P.SL2P(sl2p_inp,variableName,imageCollectionName)

Generating sl2p input data flag
Run SL2P...
SL2P start: 2025-11-04 14:20:06.668795
SL2P end: 2025-11-04 14:20:11.219135
Generating sl2p output product flag
Done


In [50]:
# Define the output path variables
output_dir = r'C:\Users\Leoun\Work\SL2P-FORCE-main\output'
tile_basename = os.path.splitext(os.path.basename(single_tif_path))[0]
imageName = os.path.join(output_dir, tile_basename) # Create a base name to replace the SAFE filename

In [51]:
profile = s2['profile'].copy()
profile.update({
    'count': 4, 
    'dtype': rasterio.float32, # Use float32 for all 4 bands
    'driver': 'GTiff'
}) 

flag_inp = varmap['sl2p_inputFlag'].astype(numpy.float32)
flag_out = varmap['sl2p_outputFlag'].astype(numpy.float32)

output_path = imageName + f"_{variableName}_PRODUCT.tif"

# 4. Write all 4 layers
with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write(varmap[variableName].astype(numpy.float32), 1)
    dst.write(varmap[variableName+'_uncertainty'].astype(numpy.float32), 2)
    dst.write(flag_inp, 3) # Write the converted input flag
    dst.write(flag_out, 4)  # Write the converted output flag

print(f"✅ 4-Band product saved successfully to: {output_path}")

✅ 4-Band product saved successfully to: C:\Users\Leoun\Work\SL2P-FORCE-main\output\test\20190726_LEVEL2_SEN2A_BOA_Albedo_PRODUCT.tif


In [ ]:
### Single FORCE TIF + SAFE S2 Angles ###

In [2]:
from tools.read_sentinel2_force_image import read_s2_force, read_single_tif_xml_angles

In [3]:
variableName = ''
# LAI ; fCOVER ; fAPAR ; CCC ; CWC ; Albedo
imageCollectionName = "S2_SINGLE_TIF"

In [4]:
# 1. Path to the multi-band TIF (Spectral data)
single_tif_path = r'C:\Users\Leoun\Work\safe_equi\20190726_LEVEL2_SEN2A_BOA.tif' 

# 2. Path to the Sentinel-2 .SAFE directory (Angular data source)
safe_angles_dir = r'C:\Users\Leoun\Work\SL2P-PYTHON-main\S2A_MSIL2A_20190726T102031_N0500_R065_T33UVU_20230509T235130.SAFE' # **Adjust to the root .SAFE folder**
tile_basename = os.path.splitext(os.path.basename(single_tif_path))[0]

In [5]:
print(f"Reading TIF bands: {single_tif_path}")
print(f"Reading XML angles from SAFE directory: {safe_angles_dir}")

# *** NEW XML HYBRID READER CALL ***
s2 = read_single_tif_xml_angles(single_tif_path, safe_angles_dir)

Reading TIF bands: C:\Users\Leoun\Work\safe_equi\20190726_LEVEL2_SEN2A_BOA.tif
Reading XML angles from SAFE directory: C:\Users\Leoun\Work\SL2P-PYTHON-main\S2A_MSIL2A_20190726T102031_N0500_R065_T33UVU_20230509T235130.SAFE
Reading and upscaling full 10m angle grid from SAFE XML...
Reading Sentinel-2 image


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:40<00:00,  5.77s/it]


Applying Pixel Clip: Row=0:3000, Col=0:3000


In [6]:
# prepare SL2P input
sl2p_inp = SL2P.prepare_sl2p_inp(s2, variableName, imageCollectionName)

Skipping resampling: Angle shapes already matched (e.g., FORCE TIF).
Computing cosSZA, cosVZA and cosRAA
Scaling Sentinel-2 bands
Selecting sl2p input bands

--- Stacking Input Arrays ---
Done!


In [7]:
varmap=SL2P.SL2P(sl2p_inp,variableName,imageCollectionName)

Generating sl2p input data flag
Run SL2P...
SL2P start: 2025-11-13 16:47:58.584995
SL2P end: 2025-11-13 16:47:59.589556
Generating sl2p output product flag
Done


In [8]:
output_dir = r'C:\Users\Leoun\Work\SL2P-FORCE-main\output\'
imageName = os.path.join(output_dir, tile_basename) # Create a base name to replace the SAFE filename

In [9]:
profile = s2['profile'].copy()
profile.update({
    'count': 4, 
    'dtype': rasterio.float32, # Use float32 for all 4 bands
    'driver': 'GTiff'
}) 

flag_inp = varmap['sl2p_inputFlag'].astype(numpy.float32)
flag_out = varmap['sl2p_outputFlag'].astype(numpy.float32)

output_path = imageName + f"_{variableName}_PRODUCT.tif"

# 4. Write all 4 layers
with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write(varmap[variableName].astype(numpy.float32), 1)
    dst.write(varmap[variableName+'_uncertainty'].astype(numpy.float32), 2)
    dst.write(flag_inp, 3) # Write the converted input flag
    dst.write(flag_out, 4)  # Write the converted output flag

print(f"✅ 4-Band product saved successfully to: {output_path}")

✅ 4-Band product saved successfully to: C:\Users\Leoun\Work\SL2P-FORCE-main\output\xml_fix2\20190726_LEVEL2_SEN2A_BOA_LAI_PRODUCT.tif
